<a href="https://colab.research.google.com/github/IdaCy/jailbreak-alert-research/blob/anders_zach_new/notebooks/code_chameleon_fast_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Check Python version (optional):
import sys
print("Python version:", sys.version)

# Get installations
!pip install --quiet torch numpy matplotlib scikit-learn pandas
!pip install --quiet huggingface_hub transformers

import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# If you want to check GPU usage:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Python version: 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.5 MB/s eta 0:00:00
Using device: cuda


In [45]:
import torch
torch.cuda.empty_cache()
%ls

RuntimeError: CUDA error: an illegal instruction was encountered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

# After running this cell, follow the link to grant Colab access to your Google Drive.

Mounted at /content/drive


In [47]:
%ls

In [4]:
!git clone https://github.com/IdaCy/jailbreak-alert-research.git
%cd jailbreak-alert-research

Cloning into 'jailbreak-alert-research'...
remote: Enumerating objects: 1700, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 1700 (delta 16), reused 26 (delta 9), pack-reused 1662 (from 2)
Receiving objects: 100% (1700/1700), 1.37 GiB | 40.25 MiB/s, done.
Resolving deltas: 100% (455/455), done.
Updating files: 100% (856/856), done.
/content/jailbreak-alert-research


In [5]:
!git checkout "anders_zach_new"

Branch 'anders_zach_new' set up to track remote branch 'anders_zach_new' from 'origin'.
Switched to a new branch 'anders_zach_new'


In [6]:
!pip install huggingface_hub --quiet

from huggingface_hub import notebook_login

# This will prompt you in Colab to enter your HF token or log in directly
notebook_login()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# Load Model and Tokeniser here first time around

In [7]:
import logging
logging.basicConfig(level=logging.INFO)

from functions.load_csv_prompts import load_prompts
from functions.csv_inference import load_model_and_tokenizer, run_inference

# 1. Load the model/tokenizer once
tokenizer, model = load_model_and_tokenizer(
    model_name="google/gemma-2-9b-it",
    use_bfloat16=True,
    hf_token=None
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [8]:
# Save model and tokenizer to Google Drive
SAVE_PATH = "/content/drive/MyDrive/models/gemma-2-9b-it"
!mkdir -p $SAVE_PATH

print(f"Saving model and tokenizer to {SAVE_PATH}...")
model.save_pretrained(SAVE_PATH)
tokenizer.save_pretrained(SAVE_PATH)
print("Save complete!")


Saving model and tokenizer to /content/drive/MyDrive/models/gemma-2-9b-it...


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2810: UserWarning: Attempting to save a model with offloaded modules. Ensure that unallocated cpu memory exceeds the `shard_size` (5GB default)
  warnings.warn(


Saving checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Save complete!


# In future, skip above, load model here

In [7]:
#in future use this to load the model and tokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer

LOAD_PATH = "/content/drive/MyDrive/models/gemma-2-9b-it"
print("Loading saved model from Google Drive...")
model = AutoModelForCausalLM.from_pretrained(
    LOAD_PATH,
    device_map="auto",
    torch_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(LOAD_PATH)
print("Model loaded successfully!")

Loading saved model from Google Drive...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Model loaded successfully!


In [37]:
import os
from functions.load_csv_prompts import load_prompts
print("hello")
# Verify current working directory and file existence
print("Current working directory:", os.getcwd())
csv_path = "data/code_chameleon/formatted_none_820jb.csv"
print("Checking for file at:", os.path.abspath(csv_path))
print("File exists:", os.path.exists(csv_path))

# 2. Load prompts from CSV
prompts = load_prompts(
    csv_path=csv_path,
    num_samples=None
)

# Force print output
print(f"Number of prompts loaded: {len(prompts)}")
print("First prompt sample:", prompts[0][:100] + "..." if prompts else "No prompts loaded")

# Create output directory
%mkdir -p output/extractions/gemma-2-9b-it

In [44]:
print("hello")

In [9]:
### just some logging ###

# Create (or get) a logger and set the level
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# OPTIONAL: if you want to clear any existing handlers, do:
# logger.handlers = []

# Create a console handler that prints to stdout
console_handler = logging.StreamHandler(sys.stdout)
console_handler.setLevel(logging.INFO)

# Create a formatter and set it for the console handler
formatter = logging.Formatter("[%(asctime)s] [%(levelname)s] %(message)s")
console_handler.setFormatter(formatter)

# Add the handler to the logger
logger.addHandler(console_handler)


NameError: name 'logging' is not defined

In [32]:
import os
import logging

# Create log directories if they don't exist
os.makedirs("logs", exist_ok=True)
os.makedirs("output/extractions/gemma-2-9b-it", exist_ok=True)

# Set up basic logging
logging.basicConfig(level=logging.INFO)


from functions.csv_inference import run_inference  # Changed from inference to csv_inference


# 3. Run inference & capture activations
run_inference(
    model=model,
    tokenizer=tokenizer,
    prompts=prompts,  # Changed from prompts_bad to match our loaded variable
    output_dir="output/extractions/gemma-2-9b-it",  # Corrected model directory
    batch_size=2,  # Reduced for CUDA stability
    max_seq_length=2048,  # Required parameter
    max_new_tokens=50,
    extract_hidden_layers=[0,5,10,15,20,25],
    extract_attention_layers=[0,5,10,15,20,25],
    top_k_logits=10,
    log_file="logs/inference.log",  # Required parameter
    error_log="logs/errors.log"  # Required parameter
)





Streaming output truncated to the last 5000 lines.
  File "/content/jailbreak-alert-research/functions/csv_inference.py", line 150, in capture_activations
    input_ids = encodings["input_ids"].cuda()
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: CUDA error: an illegal instruction was encountered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

ERROR:functions.csv_inference:Error at batch 17: CUDA error: an illegal instruction was encountered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
Traceback (most recent call last):
  File "/content/jailbreak-alert-research/functions/csv_inference.py

In [ ]:
LOCAL_PATH = "output/extractions/gemma-2-9b-it"
GDRIVE_PATH = "/content/drive/MyDrive/jailbreak_storing/output/extractions/gemma-2-9b-it"

!mkdir -p "$GDRIVE_PATH"
!cp -r "$LOCAL_PATH"/* "$GDRIVE_PATH"

print("got .pt files over to:", GDRIVE_PATH)

In [ ]:
%ls GDRIVE_PATH
%mkdir output/extractions/gemma3-4bit/attack_jailbreak

In [ ]:
# 2. Load one set of prompts
prompts_file = "data/renellm/attacks_all.json"
prompts_jb = load_json_prompts(
    prompts_file,
    prompt_key="attack_jailbreak",
    num_samples=None
)

In [ ]:
# 3. Run inference & capture activations
run_inference(
    model,
    tokenizer,
    prompts_bad,
    output_dir="output/extractions/gemma3-4bit/attack_jailbreak",
    batch_size=8,
    max_new_tokens=50,
    extract_hidden_layers=[0,5,10,15,20,25],
    extract_attention_layers=[0,5,10,15,20,25],
    top_k_logits=10
)

In [ ]:
LOCAL_PATH = "output/extractions/gemma3-4b/attack_jailbreak"
GDRIVE_PATH = "/content/drive/MyDrive/jailbreak_storing/output/extractions/gemma3-4b/attack_jailbreak"

!mkdir -p "$GDRIVE_PATH"
!cp -r "$LOCAL_PATH"/* "$GDRIVE_PATH"

print("got .pt files over to:", GDRIVE_PATH)

In [ ]:
%ls /content/drive/MyDrive/jailbreak_storing/output/extractions/gemma3-4b/
%ls /content/drive/MyDrive/jailbreak_storing/output/extractions/gemma3-4b/attack_jailbreak

In [ ]:
import sys
sys.path.append(".")

from functions.read_predictions import read_predictions

# Then call it:
preds = read_predictions(
    read_output_dir="output/extractions/gemma3-4b/attack",
    max_predictions=50,
    write_predictions_file="logs/pred_out.txt",
    log_file="logs/readpred.log"
)

print("Number of preds:", len(preds))
print("First 3 preds:", preds[:15])